In [16]:
from experta import *
from experta.fact import *
import random

NERD = True

In [17]:
class WinTotals(Fact):
    human = Field(int, default=0)
    computer = Field(int, default=0)
    ties = Field(int, default=0)


class Results(Fact):
    winner = Field(str, mandatory=True)
    loser = Field(str, mandatory=True)
    why = Field(str, mandatory=True)


class ValidAnswer(Fact):
    answer = Field(str, mandatory=True)
    key = Field(str, mandatory=True)


class Action(Fact):
    pass


class HumanChoice(Fact):
    pass


class ComputerChoice(Fact):
    pass

In [18]:
class RockPaperScissors(KnowledgeEngine):
    def yes_or_no(self, question):
        return input(question).upper().startswith('Y')
    
    @DefFacts()
    def game_rules(self, is_nerd=False):
        """Declare game rules and valid input keys for the user."""
        self.valid_answers = dict()
        
        yield Results(winner='石头', loser='剪刀', why='石头比剪刀硬')
        yield Results(winner='布', loser='石头', why='布包住了石头')
        yield Results(winner='剪刀', loser='布', why='剪刀剪开了布')
        yield ValidAnswer(answer='石头', key='s')
        yield ValidAnswer(answer='布', key='b')
        yield ValidAnswer(answer='剪刀', key='j')

        if is_nerd:
            pass
    
    @Rule()
    def startup(self):
        print("来玩个游戏!")
        print("你来选择 石头, 布, 或者剪刀,")
        print("来比比谁比较厉害！")
        self.declare(WinTotals(human=0, computer=0, ties=0))
        self.declare(Action('get-human-move'))
    
    @Rule(NOT(Action()),
          ValidAnswer(answer=MATCH.answer,
                      key=MATCH.key))
    def store_valid_answers(self, answer, key):
        self.valid_answers[key] = answer

    #
    # HUMAN MOVE RULES
    #
    @Rule(Action('get-human-move'))
    def get_human_move(self):
        question = ", ".join(
            "{name} ({key})".format(
                name=a[1].title(), key=a[0].upper())
            for a in self.valid_answers.items()) + '? '
        res = input(question).lower()
        self.declare(HumanChoice(res))
    
    @Rule(AS.f1 << HumanChoice(MATCH.choice),
          ValidAnswer(answer=MATCH.answer,
                      key=MATCH.choice),
          AS.f2 << Action('get-human-move'))
    def good_human_move(self, f1, f2, answer):
        self.retract(f1)
        self.retract(f2)
        self.declare(HumanChoice(answer))
        self.declare(Action('get-computer-move'))
    
    @Rule(AS.f1 << HumanChoice(MATCH.choice),
          NOT(ValidAnswer(key=MATCH.choice)),
          AS.f2 << Action('get-human-move'))
    def bad_human_move(self, f1, f2, choice):
        print("抱歉 %s 不是正确的输入" % choice)
        self.retract(f1)
        self.retract(f2)
        self.declare(Action('get-human-move'))
    
    #
    # COMPUTER MOVE RULES
    #
    @Rule(AS.f1 << Action('get-computer-move'))
    def get_computer_move(self, f1):
        choice = random.choice(list(self.valid_answers.values()))
        self.retract(f1)
        self.declare(ComputerChoice(choice))
        self.declare(Action('determine-results'))

    #
    # WIN DETERMINATION RULES
    #
    @Rule(AS.f1 << Action('determine-results'),
          AS.f2 << ComputerChoice(MATCH.cc),
          AS.f3 << HumanChoice(MATCH.hc),
          AS.w << WinTotals(computer=MATCH.cw),
          Results(winner=MATCH.cc,
                  loser=MATCH.hc,
                  why=MATCH.explanation))
    def computer_wins(self, f1, f2, f3, w, cw, explanation):
        self.retract(f1)
        self.retract(f2)
        self.retract(f3)
        self.modify(w, computer=cw + 1)
        print("电脑赢了!", explanation)
        self.declare(Action('determine-play-again'))
        
    @Rule(AS.f1 << Action('determine-results'),
          AS.f2 << ComputerChoice(MATCH.cc),
          AS.f3 << HumanChoice(MATCH.hc),
          'w' << WinTotals(human=MATCH.hw),
          Results(winner=MATCH.hc,
                  loser=MATCH.cc,
                  why=MATCH.explanation))
    def humans_wins(self, f1, f2, f3, w, hw, explanation):
        self.retract(f1)
        self.retract(f2)
        self.retract(f3)
        self.modify(w, human=hw + 1)
        print("你赢了!", explanation)
        self.declare(Action('determine-play-again'))
        
    @Rule(AS.f1 << Action('determine-results'),
          AS.f2 << ComputerChoice(MATCH.cc),
          AS.f3 << HumanChoice(MATCH.cc),
          AS.w << WinTotals(ties=MATCH.nt))
    def tie(self, f1, f2, f3, w, nt):
        self.retract(f1)
        self.retract(f2)
        self.retract(f3)
        self.modify(w, ties=nt + 1)
        print("平局！")
        self.declare(Action('determine-play-again'))
    
    #
    # PLAY AGAIN RULE
    #
    @Rule(AS.f1 << Action('determine-play-again'),
          WinTotals(computer=MATCH.ct,
                    human=MATCH.ht,
                    ties=MATCH.tt))
    def play_again(self, f1, ct, ht, tt):
        self.retract(f1)
        if not self.yes_or_no("还要玩吗（Y/N）?"):
            print("你总共赢了", ht, "次.")
            print("电脑总共赢了", ct, "次.")
            print("一共平局了", tt, "次.")
            self.halt()
        else:
            self.declare(Action('get-human-move'))

In [19]:
rps = RockPaperScissors()

In [20]:
rps = RockPaperScissors()
rps.reset()
rps.run()

来玩个游戏!
你来选择 石头, 布, 或者剪刀,
来比比谁比较厉害！
剪刀 (J), 布 (B), 石头 (S)? J
平局！
还要玩吗（Y/N）?Y
剪刀 (J), 布 (B), 石头 (S)? S
平局！
还要玩吗（Y/N）?Y
剪刀 (J), 布 (B), 石头 (S)? b
平局！
还要玩吗（Y/N）?y
剪刀 (J), 布 (B), 石头 (S)? j
你赢了! 剪刀剪开了布
还要玩吗（Y/N）?y
剪刀 (J), 布 (B), 石头 (S)? s
电脑赢了! 布包住了石头
还要玩吗（Y/N）?n
你总共赢了 1 次.
电脑总共赢了 1 次.
一共平局了 3 次.
